In [1]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [2]:
# 人口構成のCSVファイルの読み込み
df = pd.read_csv("population_data/japan_population.csv")
df.head()

,集計年,都道府県コード,都道府県名,総人口
0,1960,1,北海道,5039206.0
1,1960,2,青森県,1426606.0
2,1960,3,岩手県,1448517.0
3,1960,4,宮城県,1743195.0
4,1960,5,秋田県,1335580.0


In [3]:
# 1960年から2015年までの関東地方各県の総人口数データを機械学習にかける。
# ある年の都道府県コードをカテゴリ値 とした上で ダミー変数化したものと総人口数を説明変数X、その翌年の総人口数を目的変数yに設定する
# （Xは縦56×7行の横7列の二次元配列、yは縦56×7の長さの1次元配列としてndarrayを作成する）
X = np.zeros((56 * 7,7), dtype=np.uint32)
y = np.zeros(56 * 7, dtype=np.uint32)

cnt = 0
for i in range(56 * 47):
    pref_id = df.iloc[i, 1]
    population = df.iloc[i, 3]
    next_population = df.iloc[i + 47, 3]

    if pref_id >= 8 and pref_id <= 14:    #pref_id 8(茨城県)から、pref_id 14(神奈川県)を計測データXに格納する。  
        if pref_id < 14:                  #pref_id 14(神奈川県)は、ダミー変数化の際に1としない。(理由は、上部のカリキュラムで述べています。)  
            X[cnt][pref_id - 8] = 1       #計測データXに都県のダミーデータを格納する。値は1とする。8~13のpref_idを0~5として番号を振りなおす。  

        X[cnt][6] = population            #計測データXの7番目の要素は、その年の人口とする。  
        y[cnt] = next_population          #教師データyは、翌年の人口とする。  
        cnt += 1                          #カウンタ変数cntをカウントアップする。  

# Xを確認
X[0:10]

array([[      1,       0,       0,       0,       0,       0, 2047024],
       [      0,       1,       0,       0,       0,       0, 1513624],
       [      0,       0,       1,       0,       0,       0, 1578476],
       [      0,       0,       0,       1,       0,       0, 2430871],
       [      0,       0,       0,       0,       1,       0, 2306010],
       [      0,       0,       0,       0,       0,       1, 9683802],
       [      0,       0,       0,       0,       0,       0, 3443176],
       [      1,       0,       0,       0,       0,       0, 2053000],
       [      0,       1,       0,       0,       0,       0, 1513000],
       [      0,       0,       1,       0,       0,       0, 1581000]],
      dtype=uint32)

In [4]:
# yを確認
y[0:10]

array([2053000, 1513000, 1581000, 2497000, 2356000, 9967000, 3606000,
       2056000, 1513000, 1584000], dtype=uint32)

In [5]:
# 1960年から2009年までを訓練データ、
# 2010年以降のデータをテストデータとして分割する
X_train = X[:350]
X_test = X[350:]
y_train = y[:350]
y_test = y[350:]

In [6]:
# LinearRegressionで回帰モデルを作成
model1 = LinearRegression()
model1.fit(X_train, y_train)

LinearRegression()

In [7]:
# LinearRegressionで予測実行
y_pred1 = model1.predict(X_test)

In [8]:
# LinearRegressionの予測結果の表示
y_pred1 = y_pred1.astype(np.uint32)
y_pred1

array([ 2976504,  2011342,  2011854,  7234625,  6247455, 13182582,
        9095368,  2967043,  2003902,  2005009,  7248612,  6248144,
       13219973,  9106668,  2954454,  1996155,  1998231,  7255390,
        6231681, 13254834,  9116352,  2944770,  1990345,  1990484,
        7267011,  6232650, 13325526,  9129909,  2935086,  1984534,
        1983705,  7285410,  6240397, 13414617,  9148308,  2925379,
        1978971,  1978006,  7304327,  6253631, 13527211,  9170788],
      dtype=uint32)

In [9]:
# 正解の表示
y_test

array([ 2960000,  2000000,  2001000,  7209000,  6217000, 13198000,
        9060000,  2947000,  1992000,  1994000,  7216000,  6200000,
       13234000,  9070000,  2937000,  1986000,  1986000,  7228000,
        6201000, 13307000,  9084000,  2927000,  1980000,  1979000,
        7247000,  6209000, 13399000,  9103000,  2916976,  1974255,
        1973115,  7266534,  6222666, 13515271,  9126214,  2905000,
        1966000,  1967000,  7289000,  6236000, 13624000,  9145000],
      dtype=uint32)

In [10]:
# RandomForestRegressorで回帰モデルを作成
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)

RandomForestRegressor()

In [11]:
# RandomForestRegressorで予測実行
y_pred2 = model2.predict(X_test)

# RandomForestRegressorの予測結果の表示
y_pred2 = y_pred2.astype(np.uint32)
y_pred2

array([ 2970275,  2011085,  2012014,  7273562,  6279061, 13104756,
        9029201,  2968413,  2003547,  2004047,  7280081,  6279061,
       13104756,  9029201,  2960042,  1996387,  2001824,  7280081,
        6265310, 13104756,  9029201,  2953579,  1991133,  1990800,
        7284181,  6265310, 13104756,  9029201,  2947744,  1986677,
        1985112,  7316052,  6279061, 13104756,  9029201,  2942883,
        1983725,  1983560,  7316052,  6299021, 13104756,  9029201],
      dtype=uint32)

In [12]:
# SVRで回帰モデルを作成
model3 = SVR()
model3.fit(X_train, y_train)

SVR()

In [13]:
# SVRで予測実行
y_pred3 = model3.predict(X_test)

# SVRの予測結果の表示
y_pred3 = y_pred3.astype(np.uint32)
y_pred3

array([3928213, 3928193, 3928193, 3928350, 3928321, 3928364, 3928382,
       3928213, 3928192, 3928192, 3928350, 3928321, 3928364, 3928383,
       3928213, 3928192, 3928192, 3928350, 3928320, 3928363, 3928383,
       3928212, 3928192, 3928192, 3928350, 3928320, 3928362, 3928383,
       3928212, 3928192, 3928192, 3928351, 3928321, 3928361, 3928383,
       3928212, 3928192, 3928192, 3928351, 3928321, 3928359, 3928383],
      dtype=uint32)